In [3]:
import tensorflow
import pandas
import numpy
import glob
import PIL
from PIL import Image
import os
import cv2
from tensorflow import keras
from keras.utils import get_file
tensorflow.__version__

'2.10.0'

In [4]:
from keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Input, Concatenate
from keras.models import Model
from keras.applications import ResNet50

In [5]:
def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2,2), strides = 2, padding = "same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x


def build_resnet50_unet(input_shape):
    """Input fot model"""
    inputs = Input(input_shape)

    "load pre-trained model ResNet50"
    resnet50 = ResNet50(include_top = False, weights= "imagenet",input_tensor = inputs)
    # resnet50.summary(line_length=None , layer_range=None)

    """Encoder"""
    e1 = resnet50.get_layer("input_1").output  # out 512
    e2 = resnet50.get_layer("conv1_conv").output #256
    e3 = resnet50.get_layer("conv2_block3_out").output #128
    e4 = resnet50.get_layer("conv3_block4_out").output #64

    """ connect"""
    bridge = resnet50.get_layer("conv4_block6_out").output


    """Decoder"""
    d1 = decoder_block(bridge, e4, 512) # 64
    d2 = decoder_block(d1, e3, 256) #128
    d3 = decoder_block(d2, e2, 128) #256
    d4 = decoder_block(d3, e1, 63) #512


    """Output"""
    outputs = Conv2D(3 , 1, padding = "same", activation = "softmax")(d4)
    model = Model(inputs, outputs, name="fine-tune_unet")
    return model



In [6]:
if __name__ == "__main__":
    input_shape = (128,128,3)
    model = build_resnet50_unet(input_shape)
    model.summary()

Model: "fine-tune_unet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 134, 134, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 64, 64, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 64, 64, 64)   256         ['conv1_conv[0][0]']

In [7]:
inputs = Input((128,128,3))

"load pre-trained model ResNet50"
resnet50 = ResNet50(include_top = False, weights= "imagenet",input_tensor = inputs)
resnet50.summary(line_length=None , layer_range=None)

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 134, 134, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 64, 64, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 64, 64, 64)   256         ['conv1_conv[0][0]']      

In [8]:
img_file = './model_resnet.png'

tensorflow.keras.utils.plot_model(model, to_file=img_file, show_shapes=True, show_layer_names=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [9]:
model.save('model/resnet_unet.keras')
model.save('model/resnet_unet.h5')

In [10]:
# keras.utils import custom_object_scope

# with custom_object_scope({'dice_coefficients_loss': dice_coefficients_loss}):
#     model.save('model2.keras')

In [11]:
# custom_objects = {'dice_coefficients_loss': loss}
# model = tensorflow.keras.models.load_model('/kaggle/input/modell/model2.keras', custom_objects=custom_objects)